In [25]:
import pandas as pd
import pickle

# 1. Prepare

## 1.1. Load learned models from pickle

In [2]:
with open('pickles/predict_rental_price.pkl', 'rb') as f:
    regressor_rental = pickle.load(f) 
with open('pickles/predict_rental_price_scaler.pkl', 'rb') as f:
    scaler_rental = pickle.load(f) 
    
with open('pickles/predict_sale_price.pkl', 'rb') as f:
    regressor_sale = pickle.load(f) 
with open('pickles/predict_sale_price_scaler.pkl', 'rb') as f:
    scaler_sale = pickle.load(f) 

## 1.2. Build functions capable to use trained model

In [3]:
def predict_rental(test):
    
    test_ = scaler_rental.transform(test)
    display(test)

    print("For an apartment with area = {} m^2, {} room(s), build in {} and located at ({}, {}), \npredicted rental price is {} zł.".
          format(test['Powierzchnia'].iloc[0], test['Liczba pokoi'].iloc[0], test['Rok budowy'].iloc[0]
                 , test['φ'].iloc[0], test['λ'].iloc[0], round(regressor_rental.predict(test_)[0])))
    
    
def predict_sale(test):

    test_ = scaler_sale.transform(test)
    display(test)

    print("For an apartment with area = {} m^2, {} room(s), build in {} and located at ({}, {}), \npredicted sale price is {} zł.".
          format(test['Powierzchnia'].iloc[0], test['Liczba pokoi'].iloc[0], test['Rok budowy'].iloc[0]
                 , test['φ'].iloc[0], test['λ'].iloc[0], round(regressor_sale.predict(test_)[0])))

## 1.3. Test on some hand-typed data

In [4]:
test_1 = pd.DataFrame([[45, 2, 52.197751, 20.964537, 2018, 25]], columns=['Powierzchnia', 'Liczba pokoi', 'φ', 'λ', 'Rok budowy', 'Czas dojazdu do centrum'])
test_2 = pd.DataFrame([[68.80, 3, 52.147, 21.047, 1980, 29]], columns=['Powierzchnia', 'Liczba pokoi', 'φ', 'λ', 'Rok budowy', 'Czas dojazdu do centrum'])

In [5]:
predict_rental(test_1)

,Powierzchnia,Liczba pokoi,φ,λ,Rok budowy,Czas dojazdu do centrum
0,45,2,52.197751,20.964537,2018,25


For an apartment with area = 45 m^2, 2 room(s), build in 2018 and located at (52.197751, 20.964537), 
predicted rental price is 3052.0 zł.


In [6]:
predict_sale(test_2)

,Powierzchnia,Liczba pokoi,φ,λ,Rok budowy,Czas dojazdu do centrum
0,68.8,3,52.147,21.047,1980,29


For an apartment with area = 68.8 m^2, 3 room(s), build in 1980 and located at (52.147, 21.047), 
predicted sale price is 654283.0 zł.


# 2. Test

## 2.1. Read input data from user

In [74]:
import geopandas as gpd
import folium
from folium import Marker, Choropleth
from IPython.display import display
from ipywidgets import interact, interactive, FloatSlider, IntSlider, IntRangeSlider

In [11]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## 2.2. Read districts shape data from file

In [26]:
geo = gpd.read_file("districts/warszawa_dzielnice.shp").to_crs({'init': 'epsg:4326'})
geo = geo.drop(0).reset_index().drop(columns=['index', 'cartodb_id'])

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


# 2.3. Pick user input, show location on the interactive map and calculate price

In [96]:
def visualise_price(kind='sale', latitude=52.15, longitude=21.05, area=45, rooms=2, year=2018, time=25):
    
    # Create a base map
    m_1 = folium.Map(location=[52.2323, 21.0000], tiles='cartodbpositron', zoom_start=10)

    # Add marker indicating user's location to the map
    Marker([latitude, longitude]).add_to(m_1)
    
    # Add district bondaries to the map
    Choropleth(geo_data=geo.__geo_interface__).add_to(m_1)

    # Display the map
    display(embed_map(m_1, 'maps/m_1.html'))
    
    
    data = pd.DataFrame([[area, rooms, latitude, longitude, year, time]]
                          , columns=['Powierzchnia', 'Liczba pokoi', 'φ', 'λ', 'Rok budowy', 'Czas dojazdu do centrum'])
    if kind == 'sale':
        predict_sale(data)
    else:
        predict_rental(data)
    
    
interactive(visualise_price, {'manual': True}, kind=(['sale', 'rental']), latitude=(52.08, 52.36, 0.01), longitude=(20.85, 21.3, 0.01), 
           area=(10, 150, 1), rooms=(1,8,1), year=(1800, 2020, 1), time=(5, 70, 1))

interactive(children=(Dropdown(description='kind', options=('sale', 'rental'), value='sale'), FloatSlider(valu…

## One last comment:
Parameter "`time to travel to city center`"is depended on `location coordinates`, but for simplicity sake it was made as user input.